In [1]:
from itertools import product
from collections import defaultdict

import numpy as np
from scipy.spatial.distance import euclidean
import pulp
import gensim

Using TensorFlow backend.


In [2]:
def tokens_to_fracdict(tokens):
    cntdict = defaultdict(lambda : 0)
    for token in tokens:
        cntdict[token] += 1
    totalcnt = sum(cntdict.values())
    return {token: float(cnt)/totalcnt for token, cnt in cntdict.items()}

In [3]:
def word_mover_distance_probspec(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=None):
    all_tokens = list(set(first_sent_tokens+second_sent_tokens))
    wordvecs = {token: wvmodel[token] for token in all_tokens}

    first_sent_buckets = tokens_to_fracdict(first_sent_tokens)
    second_sent_buckets = tokens_to_fracdict(second_sent_tokens)

    T = pulp.LpVariable.dicts('T_matrix', list(product(all_tokens, all_tokens)), lowBound=0)

    prob = pulp.LpProblem('WMD', sense=pulp.LpMinimize)
    prob += pulp.lpSum([T[token1, token2]*euclidean(wordvecs[token1], wordvecs[token2])
                        for token1, token2 in product(all_tokens, all_tokens)])
    for token2 in second_sent_buckets:
        prob += pulp.lpSum([T[token1, token2] for token1 in first_sent_buckets])==second_sent_buckets[token2]
    for token1 in first_sent_buckets:
        prob += pulp.lpSum([T[token1, token2] for token2 in second_sent_buckets])==first_sent_buckets[token1]

    if lpFile!=None:
        prob.writeLP(lpFile)

    prob.solve()

    return prob

In [4]:
def word_mover_distance(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=None):
    prob = word_mover_distance_probspec(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=lpFile)
    return pulp.value(prob.objective)

In [5]:
wvmodel = gensim.models.KeyedVectors.load_word2vec_format('/Users/hok/Data/Word2Vec/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [6]:
prob = word_mover_distance_probspec(['President', 'talk', 'Chicago'], ['Presidents', 'speech', 'Illinois'], wvmodel)
print(pulp.value(prob.objective))
for v in prob.variables():
    if v.varValue!=0:
        print(v.name, '=', v.varValue)

2.88587622936
("T_matrix_('Chicago',_'Illinois')", '=', 0.33333333)
("T_matrix_('President',_'Presidents')", '=', 0.33333333)
("T_matrix_('talk',_'speech')", '=', 0.33333333)


In [7]:
prob = word_mover_distance_probspec(['physician', 'assistant'], ['doctor'], wvmodel)
print(pulp.value(prob.objective))
for v in prob.variables():
    if v.varValue!=0:
        print(v.name, '=', v.varValue)

2.8760048151
("T_matrix_('assistant',_'doctor')", '=', 0.5)
("T_matrix_('physician',_'doctor')", '=', 0.5)


In [8]:
prob = word_mover_distance_probspec(['physician', 'assistant'], ['doctor', 'assistant'], wvmodel)
print(pulp.value(prob.objective))
for v in prob.variables():
    if v.varValue!=0:
        print(v.name, '=', v.varValue)

1.00465738773
("T_matrix_('assistant',_'assistant')", '=', 0.5)
("T_matrix_('physician',_'doctor')", '=', 0.5)


In [9]:
prob = word_mover_distance_probspec(['doctors', 'assistant'], ['doctor', 'assistant'], wvmodel)
print(pulp.value(prob.objective))
for v in prob.variables():
    if v.varValue!=0:
        print(v.name, '=', v.varValue)

1.02825379372
("T_matrix_('assistant',_'assistant')", '=', 0.5)
("T_matrix_('doctors',_'doctor')", '=', 0.5)


In [10]:
prob = word_mover_distance_probspec(['doctor', 'assistant'], ['doctor', 'assistant'], wvmodel)
print(pulp.value(prob.objective))
for v in prob.variables():
    if v.varValue!=0:
        print(v.name, '=', v.varValue)

0.0
("T_matrix_('assistant',_'assistant')", '=', 0.5)
("T_matrix_('doctor',_'doctor')", '=', 0.5)


In [11]:
prob = word_mover_distance_probspec(['Washington', 'metro', 'train'], ['bubble', 'milk', 'tea'], wvmodel)
print(pulp.value(prob.objective))
for v in prob.variables():
    if v.varValue!=0:
        print(v.name, '=', v.varValue)

4.05659282125
("T_matrix_('Washington',_'tea')", '=', 0.33333333)
("T_matrix_('metro',_'bubble')", '=', 0.33333333)
("T_matrix_('train',_'milk')", '=', 0.33333333)


In [12]:
prob = word_mover_distance_probspec(['colon', 'cancer', 'research', 'patient'], ['cancer', 'clinical', 'study', 'optics', 'informatics'], wvmodel)
print(pulp.value(prob.objective))
for v in prob.variables():
    if v.varValue!=0:
        print(v.name, '=', v.varValue)

2.78057779074
("T_matrix_('cancer',_'cancer')", '=', 0.2)
("T_matrix_('cancer',_'informatics')", '=', 0.05)
("T_matrix_('colon',_'informatics')", '=', 0.05)
("T_matrix_('colon',_'optics')", '=', 0.2)
("T_matrix_('patient',_'clinical')", '=', 0.2)
("T_matrix_('patient',_'informatics')", '=', 0.05)
("T_matrix_('research',_'informatics')", '=', 0.05)
("T_matrix_('research',_'study')", '=', 0.2)


In [13]:
prob = word_mover_distance_probspec(['China', 'Beijing'], ['France', 'Paris'], wvmodel)
print(pulp.value(prob.objective))
for v in prob.variables():
    if v.varValue!=0:
        print(v.name, '=', v.varValue)

3.18052220345
("T_matrix_('Beijing',_'Paris')", '=', 0.5)
("T_matrix_('China',_'France')", '=', 0.5)


In [14]:
prob = word_mover_distance_probspec(['China', 'Beijing'], ['Taiwan', 'Taipei'], wvmodel)
print(pulp.value(prob.objective))
for v in prob.variables():
    if v.varValue!=0:
        print(v.name, '=', v.varValue)

2.55220150948
("T_matrix_('Beijing',_'Taipei')", '=', 0.5)
("T_matrix_('China',_'Taiwan')", '=', 0.5)


In [15]:
prob = word_mover_distance_probspec(['China', 'Beijing'], ['Chile', 'Santiago'], wvmodel)
print(pulp.value(prob.objective))
for v in prob.variables():
    if v.varValue!=0:
        print(v.name, '=', v.varValue)

3.48362541199
("T_matrix_('Beijing',_'Santiago')", '=', 0.5)
("T_matrix_('China',_'Chile')", '=', 0.5)
